In [17]:
from os import listdir
from numpy import array
from numpy import argmax
from numpy import zeros
from pandas import DataFrame
from nltk.translate.bleu_score import corpus_bleu
from pickle import load
 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Embedding
from keras.layers.merge import concatenate
from keras.layers.pooling import GlobalMaxPooling2D
 
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# load a pre-defined list of photo identifiers
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    # process line by line
    for line in doc.split('\n'):
        # skip empty lines
        if len(line) < 1:
            continue
        # get the image identifier
        identifier = line.split('.')[0]
        dataset.append(identifier)
    return set(dataset)
 
# split a dataset into train/test elements
def train_test_split(dataset):
    # order keys so the split is consistent
    ordered = sorted(dataset)
    # return split dataset as two new sets
    return set(ordered[:100]), set(ordered[100:200])
 
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
    # load document
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        # split line by white space
        tokens = line.split()
        # split id from description
        image_id, image_desc = tokens[0], tokens[1:]
        # skip images not in the set
        if image_id in dataset:
            # store
            descriptions[image_id] = 'startseq ' + ' '.join(image_desc) + ' endseq'
    return descriptions
 
# load photo features
def load_photo_features(filename, dataset):
    # load all features
    all_features = load(open(filename, 'rb'))
    # filter features
    features = {k: all_features[k] for k in dataset}
    return features
 
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
    lines = list(descriptions.values())
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
 
# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, descriptions, photos):
    X1, X2, y = list(), list(), list()
    # walk through each image identifier
    for key, desc_list in descriptions.items():
        # walk through each description for the image
        for desc in desc_list:
            # encode the sequence
            seq = tokenizer.texts_to_sequences([desc])[0]
            # split one sequence into multiple X,y pairs
            for i in range(1, len(seq)):
                # split into input and output pair
                in_seq, out_seq = seq[:i], seq[i]
                # pad input sequence
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                # encode output sequence
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                # store
                X1.append(photos[key][0])
                X2.append(in_seq)
                y.append(out_seq)
    return array(X1), array(X2), array(y)

# load a word embedding
def load_embedding(tokenizer, vocab_size, max_length, embedding_name='word2vec_embedding.pkl'):
    # load the tokenizer
    embedding = load(open(embedding_name, 'rb'))
    dimensions = 100
    trainable = False
    # create a weight matrix for words in training docs
    weights = zeros((vocab_size, dimensions))
    # walk words in order of tokenizer vocab to ensure vectors are in the right index
    for word, i in tokenizer.word_index.items():
        if word not in embedding:
            continue
        weights[i] = embedding[word]
    layer = Embedding(vocab_size, dimensions, weights=[weights], input_length=max_length, trainable=trainable, mask_zero=True)
    return layer
 
# define the captioning model
def define_model(vocab_size, max_length, embedding_name, tokenizer=None):
    # feature extractor (encoder)
    inputs1 = Input(shape=(7, 7, 2048))
    fe1 = GlobalMaxPooling2D()(inputs1)
    fe2 = Dense(128, activation='relu')(fe1)
    fe3 = RepeatVector(max_length)(fe2)
    # embedding
    inputs2 = Input(shape=(max_length,))
    emb2=None
    if embedding_name == '':
        emb2 = Embedding(vocab_size, 50, mask_zero=True)(inputs2)
    else:
        emb2 = load_embedding(tokenizer, vocab_size, max_length)(inputs2)
    emb3 = LSTM(256, return_sequences=True)(emb2)
    emb4 = TimeDistributed(Dense(128, activation='relu'))(emb3)
    # merge inputs
    merged = concatenate([fe3, emb4])
    # language model (decoder)
    lm2 = LSTM(500)(merged)
    lm3 = Dense(500, activation='relu')(lm2)
    outputs = Dense(vocab_size, activation='softmax')(lm3)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    plot_model(model, show_shapes=True, to_file='resnet_word2_vec.png')
    return model
 
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, features, tokenizer, max_length, n_step):
    # loop until we finish training
    while 1:
        # loop over photo identifiers in the dataset
        keys = list(descriptions.keys())
        for i in range(0, len(keys), n_step):
            Ximages, XSeq, y = list(), list(),list()
            for j in range(i, min(len(keys), i+n_step)):
                image_id = keys[j]
                # retrieve photo feature input
                image = features[image_id][0]
                # retrieve text input
                desc = descriptions[image_id]
                # generate input-output pairs
                in_img, in_seq, out_word = create_sequences(tokenizer, desc, image, max_length)
                for k in range(len(in_img)):
                    Ximages.append(in_img[k])
                    XSeq.append(in_seq[k])
                    y.append(out_word[k])
            # yield this batch of samples to the model
            yield [[array(Ximages), array(XSeq)], array(y)]

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None
 
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'startseq'
    # iterate over the whole length of the sequence
    for i in range(max_length):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,sequence], verbose=0)
        # convert probability to integer
        yhat = argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += ' ' + word
        # stop if we predict the end of the sequence
        if word == 'endseq':
            break
    return in_text
 
# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()
    # step over the whole set
    for key, desc in descriptions.items():
        # generate description
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
        # store actual and predicted
        actual.append([desc.split()])
        predicted.append(yhat.split())
    # calculate BLEU score
    bleu = corpus_bleu(actual, predicted)
    return bleu

In [10]:
# load dev set
filename = '/Users/jmarcano/Downloads/Flickr8k_text/Flickr_8k.devImages.txt'
dataset = load_set(filename)
print('Dataset: %d' % len(dataset))
# train-test split
train, test = train_test_split(dataset)
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: train=%d, test=%d' % (len(train_descriptions), len(test_descriptions)))
# photo features
train_features = load_photo_features_big('features_resnet_pred.pkl', train)
test_features = load_photo_features_big('features_resnet_pred.pkl', test)
print('Photos: train=%d, test=%d' % (len(train_features), len(test_features)))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max(len(s.split()) for s in list(train_descriptions.values()))
print('Description Length: %d' % max_length)

Dataset: 1000
Descriptions: train=100, test=100
Photos: train=100, test=100
Vocabulary Size: 370
Description Length: 22
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 7, 7, 2048)   0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 22)           0                                            
__________________________________________________________________________________________________
global_max_pooling2d_4 (GlobalM (None, 2048)         0           input_7[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 22, 100)      37000       input_8[0][0]          

KeyboardInterrupt: 

In [12]:
# define experiment
model_name = 'resnet_word2_vec'
verbose = 2
n_epochs = 50
n_photos_per_update = 20
n_batches_per_epoch = int(len(train) / n_photos_per_update)
n_repeats = 3

X1train, X2train, ytrain = create_sequences(tokenizer, max_len, train_descriptions, train_features)

# run experiment
train_results, test_results = list(), list()
for i in range(n_repeats):
    # define the model
    model = define_model(vocab_size, max_length, 'word2vec_embedding.pkl', tokenizer)
    # fit model
    # 	model.fit_generator(data_generator(train_descriptions, train_features, tokenizer, max_length, n_photos_per_update), 
    #                         steps_per_epoch=n_batches_per_epoch, epochs=n_epochs, verbose=verbose)
    model.fit()
    
    # evaluate model on training data
    train_score = evaluate_model(model, train_descriptions, train_features, tokenizer, max_length)
    test_score = evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)
    # store
    train_results.append(train_score)
    test_results.append(test_score)
    print('>%d: train=%f test=%f' % ((i+1), train_score, test_score))
# save results to file
df = DataFrame()
df['train'] = train_results
df['test'] = test_results
print(df.describe())
df.to_csv(model_name+'.csv', index=False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 7, 7, 2048)   0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 22)           0                                            
__________________________________________________________________________________________________
global_max_pooling2d_6 (GlobalM (None, 2048)         0           input_11[0][0]                   
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 22, 100)      37000       input_12[0][0]                   
__________________________________________________________________________________________________
dense_18 (

Epoch 1/50
 - 6s - loss: 5.6254 - acc: 0.0622
Epoch 2/50
 - 4s - loss: 5.0515 - acc: 0.0881
Epoch 3/50
 - 4s - loss: 4.9628 - acc: 0.0929
Epoch 4/50
 - 4s - loss: 4.9188 - acc: 0.1099
Epoch 5/50
 - 4s - loss: 4.8816 - acc: 0.1119
Epoch 6/50
 - 4s - loss: 4.8446 - acc: 0.1163
Epoch 7/50
 - 4s - loss: 4.8005 - acc: 0.1163
Epoch 8/50
 - 5s - loss: 4.7446 - acc: 0.1190
Epoch 9/50
 - 5s - loss: 4.6644 - acc: 0.1209
Epoch 10/50
 - 5s - loss: 4.5798 - acc: 0.1254
Epoch 11/50
 - 4s - loss: 4.5049 - acc: 0.1326
Epoch 12/50
 - 5s - loss: 4.3975 - acc: 0.1488
Epoch 13/50
 - 4s - loss: 4.2907 - acc: 0.1659
Epoch 14/50
 - 4s - loss: 4.2004 - acc: 0.1650
Epoch 15/50
 - 4s - loss: 4.1332 - acc: 0.1722
Epoch 16/50
 - 4s - loss: 4.0665 - acc: 0.1699
Epoch 17/50
 - 5s - loss: 3.9988 - acc: 0.1695
Epoch 18/50
 - 4s - loss: 3.9337 - acc: 0.1660
Epoch 19/50
 - 4s - loss: 3.7944 - acc: 0.1874
Epoch 20/50
 - 4s - loss: 3.6266 - acc: 0.1945
Epoch 21/50
 - 4s - loss: 3.4643 - acc: 0.1954
Epoch 22/50
 - 4s - lo

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [7]:
def load_photo_features_big(filename, dataset):
    # load all features
    import joblib
    all_features = joblib.load(open(filename, 'rb'))
    # filter features
    features = {k: all_features[k] for k in dataset}
    return features

In [13]:
model.save_weights('resnet_word2_vec.h5')

In [14]:
model.save('model_resnet_word2_vec.h5')

In [15]:
model.model.save('model_model_resnet_word2_vec.h5')

AttributeError: 'Model' object has no attribute 'model'

In [24]:
# train dataset
 
# load training dataset (6K)
filename = '/Users/jmarcano/Downloads/Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features_big('features_resnet_pred.pkl', train)
print('Photos: train=%d' % len(train_features))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_len = max_length(train_descriptions)
print('Description Length: %d' % max_len)
# prepare sequences
X1train, X2train, ytrain = create_sequences(tokenizer, max_len, train_descriptions, train_features)
 
# dev dataset
 
# load test set
filename = '/Users/jmarcano/Downloads/Flickr8k_text/Flickr_8k.devImages.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features_big('features_resnet_pred.pkl', test)
print('Photos: test=%d' % len(test_features))
# prepare sequences
X1test, X2test, ytest = create_sequences(tokenizer, max_len, test_descriptions, test_features)
 
# fit model
 
# define the model
model = define_model(vocab_size, max_length, 'word2vec_embedding.pkl', tokenizer)
# define checkpoint callback
filepath = 'w2v_resnet-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

Dataset: 6000
Descriptions: train=6000
Photos: train=6000
Vocabulary Size: 3838
Description Length: 1
Dataset: 1000
Descriptions: test=1000
Photos: test=1000


TypeError: Tensors in list passed to 'values' of 'Pack' Op have types [int32, <NOT CONVERTIBLE TO TENSOR>, int32] that don't all match.

In [22]:
def max_length(descriptions):
    lines = to_lines(descriptions)
    return max(len(d.split()) for d in lines)

def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

In [ ]:
print(max_len)